In [5]:
import pandas as pd
import numpy as np

In [6]:
df=pd.read_csv('../data/min_data.csv')
df.shape

(200, 76)

In [7]:
# メタ系統の情報の削除
columns_url = [col for col in df.columns if 'url' in col]

columns_scrape = [col for col in df.columns if 'scrape' in col]
columns_scrape

columns_id = [col for col in df.columns if 'id' in col]
columns_id

description_list = ['description', 'overview', 'about', 'name']
columns_to_add = []
for col in df.columns:
    for keyword in description_list:
        if keyword in col:
            columns_to_add.append(col)
            break

num_rows = len(df)
valid_counts = df.count()
null_columns = valid_counts[valid_counts / num_rows < 0.5].index.to_list()


list = columns_url + columns_scrape +columns_id+columns_to_add +null_columns
drop_list = set(list)
drop_list

{'Unnamed: 0',
 'bathrooms',
 'calendar_last_scraped',
 'calendar_updated',
 'description',
 'host_about',
 'host_id',
 'host_identity_verified',
 'host_name',
 'host_neighbourhood',
 'host_picture_url',
 'host_thumbnail_url',
 'host_url',
 'id',
 'last_scraped',
 'listing_url',
 'name',
 'neighborhood_overview',
 'neighbourhood_group_cleansed',
 'picture_url',
 'scrape_id'}

In [8]:
num_rows = len(df)
valid_counts = df.count()
null_columns = valid_counts[valid_counts / num_rows < 0.5].index.to_list()
null_columns

['host_neighbourhood',
 'neighbourhood_group_cleansed',
 'bathrooms',
 'calendar_updated']

In [9]:
df_cleansed = df.drop(columns=drop_list, axis=1)
df_cleansed=df_cleansed.reset_index(drop=True)
df_cleansed.head()

host_df = df_cleansed[[col for col in df_cleansed.columns if 'host' in col]]

review_df = df_cleansed[[col for col in df_cleansed.columns if 'review' in col]]

state_df = df_cleansed[[col for col in df_cleansed.columns if 'availability' in col]]

keywords_to_exclude = ['host', 'review', 'availability']
# キーワードを含まない列名だけを抽出
listing_columns = [col for col in df_cleansed.columns if not any(keyword in col for keyword in keywords_to_exclude)]
# 新しいデータフレームを作成
listing_df = df_cleansed[listing_columns]

## hostテーブルの処理

In [10]:
host_df.head()

,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,2023-01-18,NaN,within an hour,100%,100%,f,2,5,"['email', 'phone']",t,2,0,2,0
1,2019-01-17,"Shibuya City, Japan",within a few hours,97%,83%,f,14,19,"['email', 'phone']",t,12,12,0,0
2,2020-05-24,NaN,NaN,NaN,100%,f,6,9,"['email', 'phone']",t,1,1,0,0
3,2020-06-23,"Tokyo, Japan",NaN,NaN,NaN,f,8,8,"['email', 'phone']",t,8,8,0,0
4,2015-10-29,"Koto City, Japan",within an hour,100%,100%,f,1,20,"['email', 'phone']",t,1,1,0,0


In [11]:
host_drop_list = ['host_verifications', 'host_has_profile_pic','host_location']
host_df_dropped = host_df.drop(columns=host_drop_list, axis=1)
host_df_dropped.head()

,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,2023-01-18,within an hour,100%,100%,f,2,5,2,0,2,0
1,2019-01-17,within a few hours,97%,83%,f,14,19,12,12,0,0
2,2020-05-24,NaN,NaN,100%,f,6,9,1,1,0,0
3,2020-06-23,NaN,NaN,NaN,f,8,8,8,8,0,0
4,2015-10-29,within an hour,100%,100%,f,1,20,1,1,0,0


In [12]:
host_df_dropped.dtypes
host_df_dropped['host_since'] = pd.to_datetime(host_df_dropped['host_since'])
host_to_process = ['host_response_rate', 'host_acceptance_rate']
for col in host_to_process:
    host_df_dropped[col]=host_df_dropped[col].str.replace('%', '').astype(float)/100

host_df_dropped.head()

,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,2023-01-18,within an hour,1.00,1.00,f,2,5,2,0,2,0
1,2019-01-17,within a few hours,0.97,0.83,f,14,19,12,12,0,0
2,2020-05-24,NaN,NaN,1.00,f,6,9,1,1,0,0
3,2020-06-23,NaN,NaN,NaN,f,8,8,8,8,0,0
4,2015-10-29,within an hour,1.00,1.00,f,1,20,1,1,0,0


## レビューテーブルの処理

In [13]:
review_df.head()

,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,0,0,2021-09-13,2021-12-26,4.60,4.60,4.80,5.00,5.0,4.80,4.60,0.23
3,7,0,0,2021-01-01,2021-11-04,5.00,4.86,5.00,5.00,5.0,4.86,4.71,0.23
4,14,9,0,2021-11-07,2023-05-09,4.36,4.29,3.93,4.36,4.5,4.29,4.14,0.70


In [14]:
review_to_process = ['first_review', 'last_review']
for col in review_to_process:
    review_df[col] = pd.to_datetime(review_df[col])

review_df.dtypes

/tmp/ipykernel_2210/3598915549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df[col] = pd.to_datetime(review_df[col])
/tmp/ipykernel_2210/3598915549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df[col] = pd.to_datetime(review_df[col])


number_of_reviews                       int64
number_of_reviews_ltm                   int64
number_of_reviews_l30d                  int64
first_review                   datetime64[ns]
last_review                    datetime64[ns]
review_scores_rating                  float64
review_scores_accuracy                float64
review_scores_cleanliness             float64
review_scores_checkin                 float64
review_scores_communication           float64
review_scores_location                float64
review_scores_value                   float64
reviews_per_month                     float64
dtype: object

## listingテーブルの処理

In [15]:
listing_df.head()

,source,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,...,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,license,instant_bookable
0,city scrape,NaN,Taito Ku,35.704772,139.778456,Room in hotel,Private room,2,1 private bath,1.0,...,1,365,1.0,1.0,365.0,365.0,1.0,365.0,Hotels and Inns Business Act | 東京都台東区台東保健所 | 3...,t
1,city scrape,"Shibuya City, Tokyo, Japan",Shibuya Ku,35.661810,139.696597,Entire rental unit,Entire home/apt,5,1 bath,2.0,...,1,365,1.0,1.0,365.0,365.0,1.0,365.0,M130004306,t
2,previous scrape,NaN,Itabashi Ku,35.746600,139.699500,Entire condo,Entire home/apt,2,1.5 baths,1.0,...,1,365,1.0,1.0,1125.0,1125.0,1.0,1125.0,M130029127,t
3,city scrape,NaN,Shinjuku Ku,35.699880,139.705050,Entire rental unit,Entire home/apt,6,1 bath,1.0,...,120,120,120.0,120.0,120.0,120.0,120.0,120.0,M130027107,t
4,city scrape,"Setagaya City, Tokyo, Japan",Setagaya Ku,35.657500,139.594850,Entire home,Entire home/apt,8,1.5 baths,2.0,...,2,1125,2.0,2.0,1125.0,1125.0,2.0,1125.0,M130019912,t


In [16]:
other_drop_list = ['source', 'neighbourhood','minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'license','amenities']

df_listing = listing_df.drop(columns=other_drop_list, axis=1)

In [17]:
df_listing.head()

,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,instant_bookable
0,Taito Ku,35.704772,139.778456,Room in hotel,Private room,2,1 private bath,1.0,1.0,"$11,500.00",1,365,t
1,Shibuya Ku,35.661810,139.696597,Entire rental unit,Entire home/apt,5,1 bath,2.0,4.0,"$41,714.00",1,365,t
2,Itabashi Ku,35.746600,139.699500,Entire condo,Entire home/apt,2,1.5 baths,1.0,2.0,"$4,000.00",1,365,t
3,Shinjuku Ku,35.699880,139.705050,Entire rental unit,Entire home/apt,6,1 bath,1.0,6.0,"$500,000.00",120,120,t
4,Setagaya Ku,35.657500,139.594850,Entire home,Entire home/apt,8,1.5 baths,2.0,6.0,"$28,833.00",2,1125,t


In [18]:
df_listing['price'] = df_listing['price'].str.replace('[$,]', '', regex=True).astype(float)
df_listing.dtypes

neighbourhood_cleansed     object
latitude                  float64
longitude                 float64
property_type              object
room_type                  object
accommodates                int64
bathrooms_text             object
bedrooms                  float64
beds                      float64
price                     float64
minimum_nights              int64
maximum_nights              int64
instant_bookable           object
dtype: object

## 利用可能テーブル

In [19]:
state_df.head()

,has_availability,availability_30,availability_60,availability_90,availability_365
0,t,10,21,38,209
1,t,9,28,32,32
2,t,0,0,0,0
3,t,29,59,89,364
4,t,0,17,47,47


In [20]:
state_df.dtypes

has_availability    object
availability_30      int64
availability_60      int64
availability_90      int64
availability_365     int64
dtype: object